In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
%matplotlib inline
from finrl.config_tickers import SP_500_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from settings import *
from pprint import pprint
import sys
import os
import itertools
import json

if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)

sys.path.append("../FinRL-Library")


In [4]:
INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']

with open("data/stock.json") as f:
    list_tickers = list(json.load(f).keys())

In [ ]:
# df = YahooDownloader(start_date = TRAIN_START_DATE,
                     # end_date = TEST_END_DATE,
                     # ticker_list = list_tickers).fetch_data()

df = pd.read_csv("data/full_data.csv", index_col=0)
df.head()

In [6]:
processed = pd.read_csv("df_processed.csv")
processed.sort_values(['date','tic'],ignore_index=True).tail(10)

,Unnamed: 0,date,open,high,low,close,volume,tic,day,macd,rsi_30,cci_30,dx_30,turbulence
251690,251690,2021-12-29,78.199997,78.639999,77.769997,73.107864,1122400,SYY,2,0.530034,53.760107,159.147158,16.522302,16.99864
251691,251691,2021-12-29,504.779999,507.220001,502.000000,489.852081,1573900,UNH,2,13.187855,68.767843,127.791454,45.467053,16.99864
251692,251692,2021-12-29,41.680000,41.820000,41.180000,37.804081,580100,VNO,2,-0.367866,47.954637,-26.616039,15.883827,16.99864
251693,251693,2021-12-29,253.639999,256.570007,253.289993,255.929993,325900,VRSN,2,4.372965,64.742318,280.638795,45.329009,16.99864
251694,251694,2021-12-29,51.450001,52.310001,51.279999,45.432167,5117300,WBA,2,0.853457,61.200644,156.908726,36.986792,16.99864
251695,251695,2021-12-29,96.650002,97.089996,96.349998,88.928246,556700,WEC,2,1.363637,58.682591,108.633868,36.199452,16.99864
251696,251696,2021-12-29,230.070007,234.360001,230.070007,204.420471,275400,WHR,2,1.604235,56.181706,69.907236,8.773698,16.99864
251697,251697,2021-12-29,165.610001,166.860001,164.940002,160.313568,773100,WM,2,0.586869,57.280689,139.208305,22.267360,16.99864
251698,251698,2021-12-29,465.000000,470.480011,462.950012,465.271881,219400,WST,2,8.414886,59.415887,188.021952,25.899134,16.99864
251699,251699,2021-12-29,124.349518,125.621361,124.019417,122.284866,2508771,ZBH,2,-1.090622,46.150884,59.872990,6.069951,16.99864


## Train/Test split

In [13]:
train = data_split(processed, TRAIN_START_DATE, TRAIN_END_DATE)
trade = data_split(processed, TEST_START_DATE, TEST_END_DATE)
print(len(train))
print(len(trade))

201400
50300


In [14]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 50, State Space: 301


In [21]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}



In [25]:
e_train_gym = StockTradingEnv(df = train, **env_kwargs)
agent = DRLAgent(e_train_gym)
if_using_a2c = True
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=50000)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| time/                 |           |
|    fps                | 59        |
|    iterations         | 100       |
|    time_elapsed       | 8         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -70.9     |
|    explained_variance | 0.0535    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -33.9     |
|    reward             | 1.0868909 |
|    std                | 0.999     |
|    value_loss         | 2.06      |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 58        |
|    iterations         | 200       |
|    time_elapsed       | 17        |
|    total_timesteps    | 1000      |
| train/    

## Trade

In [37]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 380, **env_kwargs)

df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

df_account_value.to_csv("df_account_value_a2c.csv")
df_actions.to_csv("df_actions_a2c.csv")


hit end!


In [36]:
df_account_value.head()

,date,account_value
0,2018-01-02,1.000000e+06
1,2018-01-03,1.001235e+06
2,2018-01-04,1.001629e+06
3,2018-01-05,1.003857e+06
4,2018-01-08,1.003401e+06


In [35]:
df_actions.head()

,ADM,AIG,AVB,BA,BIO,BLK,CAG,CPB,DHI,DLTR,...,SYY,UNH,VNO,VRSN,WBA,WEC,WHR,WM,WST,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,0,100,100,0,0,0,0,0,0,9,...,85,77,0,0,100,0,0,40,0,5
2018-01-03,0,100,100,0,0,0,0,0,0,9,...,85,77,0,0,100,0,0,40,0,5
2018-01-04,0,100,100,0,0,0,0,0,0,9,...,85,77,0,0,100,0,0,40,0,5
2018-01-05,0,100,100,0,0,0,0,0,0,9,...,85,77,0,0,100,0,0,40,0,5
2018-01-08,0,100,100,0,0,0,0,0,0,9,...,85,77,0,0,100,0,0,40,0,5
